Name: Ahmet Ercan Tekden

I hereby declare that I observed the honour code of the university when preparing the homework.

## Programming Homework

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [ ]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [90]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['.']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

a 0.1062437
b 0.0444502
c 0.0391600
d 0.0282947
e 0.0213084
f 0.0400793
g 0.0171783
h 0.0606047
i 0.0678165
j 0.0034660
k 0.0045451
l 0.0243019
m 0.0406429
n 0.0234882
o 0.0649920
p 0.0273498
q 0.0022208
r 0.0214068
s 0.0704687
t 0.1460781
u 0.0092399
v 0.0079497
w 0.0606385
x 0.0001107
y 0.0114638
z 0.0002911
. 0.0562102


In [71]:
randoms=np.random.randint(10,size=27)
exps=np.exp(randoms)
probabilities=(exps/np.sum(exps))

In [72]:
probabilities

array([3.04947642e-05, 3.04947642e-05, 6.12503712e-04, 2.47101633e-01,
       2.25327523e-04, 9.09036108e-02, 2.47101633e-01, 6.12503712e-04,
       1.23024659e-02, 1.66495771e-03, 8.28933633e-05, 1.66495771e-03,
       3.04947642e-05, 1.23024659e-02, 1.23024659e-02, 2.25327523e-04,
       6.12503712e-04, 8.28933633e-05, 2.47101633e-01, 3.34415695e-02,
       1.66495771e-03, 1.23024659e-02, 3.34415695e-02, 4.52582429e-03,
       3.34415695e-02, 1.66495771e-03, 4.52582429e-03])

In [73]:
np.argmax(probabilities* np.asarray(T[letter2idx['a']],dtype=float))

18

## 1

For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$


In [80]:
import numpy as np
def sample_string(x,N):
    if N==0:
        return x
    randoms=np.random.randint(10,size=27)
    exps=np.exp(randoms)
    probabilities=(exps/np.sum(exps))*np.asarray(T[letter2idx[x[-1]]],dtype=float)
    if np.argmax(probabilities)==26:
        nextCharacter='.'
    else:
        nextCharacter=chr(np.argmax(probabilities)+ord('a'))
    return sample_string(x+nextCharacter,N-1)

In [89]:
sample_string('.',10)

'.ilye.ish.s'

## 2 
Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 

In [141]:

def getAllStringProbabilities(x):
    possibleStrings=['']
    for char in x:
        nextpossibleStrings=[]
        for word in possibleStrings: 
            if char=='?':
                for i in range(26):
                    nextpossibleStrings.append(word+chr(i+ord('a')))
            else:
                nextpossibleStrings.append(word+char)
        possibleStrings=nextpossibleStrings
    return possibleStrings

In [142]:
getAllStringProbabilities('t??.')

['taa.',
 'tab.',
 'tac.',
 'tad.',
 'tae.',
 'taf.',
 'tag.',
 'tah.',
 'tai.',
 'taj.',
 'tak.',
 'tal.',
 'tam.',
 'tan.',
 'tao.',
 'tap.',
 'taq.',
 'tar.',
 'tas.',
 'tat.',
 'tau.',
 'tav.',
 'taw.',
 'tax.',
 'tay.',
 'taz.',
 'tba.',
 'tbb.',
 'tbc.',
 'tbd.',
 'tbe.',
 'tbf.',
 'tbg.',
 'tbh.',
 'tbi.',
 'tbj.',
 'tbk.',
 'tbl.',
 'tbm.',
 'tbn.',
 'tbo.',
 'tbp.',
 'tbq.',
 'tbr.',
 'tbs.',
 'tbt.',
 'tbu.',
 'tbv.',
 'tbw.',
 'tbx.',
 'tby.',
 'tbz.',
 'tca.',
 'tcb.',
 'tcc.',
 'tcd.',
 'tce.',
 'tcf.',
 'tcg.',
 'tch.',
 'tci.',
 'tcj.',
 'tck.',
 'tcl.',
 'tcm.',
 'tcn.',
 'tco.',
 'tcp.',
 'tcq.',
 'tcr.',
 'tcs.',
 'tct.',
 'tcu.',
 'tcv.',
 'tcw.',
 'tcx.',
 'tcy.',
 'tcz.',
 'tda.',
 'tdb.',
 'tdc.',
 'tdd.',
 'tde.',
 'tdf.',
 'tdg.',
 'tdh.',
 'tdi.',
 'tdj.',
 'tdk.',
 'tdl.',
 'tdm.',
 'tdn.',
 'tdo.',
 'tdp.',
 'tdq.',
 'tdr.',
 'tds.',
 'tdt.',
 'tdu.',
 'tdv.',
 'tdw.',
 'tdx.',
 'tdy.',
 'tdz.',
 'tea.',
 'teb.',
 'tec.',
 'ted.',
 'tee.',
 'tef.',
 'teg.',
 

## 3
Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.

In [176]:
def getStringProbability(x):
    prob=0
    for charIndex in range(len(x)-1):
        currChar=letter2idx[x[charIndex]]
        nextChar=letter2idx[x[charIndex+1]]
        prob=prob+ np.log(float(T[currChar][nextChar])+0.00000000000001)
    return prob

def GetBestString(X):
    possibleStrings=getAllStringProbabilities(X)
    stringProbs=[]
    for word in possibleStrings:
        prob=getStringProbability(word)
        stringProbs.append((word,prob))
    
    bestWord='.'
    bestProb=-100
    for pair in stringProbs:
        if pair[1]>bestProb:
            bestProb=pair[1]
            bestWord=pair[0]
    return bestWord,bestProb

In [ ]:
GetBestString('t??.')

Note: Normally viterbi algorithm should be used for such use case. This one can increase exceedingly fast.


## 4
Discuss how you can improve the model to get better estimations.

* We only considered p($x_i|x_{i-1}$), possibly more step transition could be used: p($x_i|x_{i-1},x_{i-2},x_{i-3}$)
* I did a simple smoothing for non existing transitions, better smoothing can be done.